In [2]:
import os
os.chdir("/home/yizhou7/mariposa")

from debugger.demo_utils import *
from debugger3 import Debugger3
from debugger.tree_node import NodeRef

from pandas import DataFrame 
import networkx as nx

In [6]:
q = "dbg/8852397c03/"
r = Reviewer2(q)
report = r.build_report()

[INFO] [init] resolved to data/projs/bench_unstable/base.z3/fs_vwasm--compiler-sandbox-queries-Compiler.Sandbox-264.smt2 
[INFO] [init] dbg root: dbg/8852397c03/ 


In [7]:
# print the instantiation report
extended = report.freq.merge(report.tested, on="qname")
extended["stabilized"] = extended["qname"].isin(report.stabilized["qname"])
extended = extended.sort_values(by="trace_count", ascending=False)
extended = extended[["qname", "trace_count", "stabilized", "action", "time", "edit_path"]]
print(tabulate(extended, headers='keys', tablefmt='psql'))

+-----+------------------+---------------+--------------+--------------+--------+-------------------------------------------------------+
|     | qname            |   trace_count | stabilized   | action       |   time | edit_path                                             |
|-----+------------------+---------------+--------------+--------------+--------+-------------------------------------------------------|
|  74 | mariposa_qid_78  |           213 | False        | inst_keep    |  0.152 | data/projs/singleton_8852397c03/base.z3/c7249e69.smt2 |
|  73 | mariposa_qid_78  |           213 | False        | inst_replace |  0.139 | data/projs/singleton_8852397c03/base.z3/10a1262a.smt2 |
| 296 | mariposa_qid_21  |           202 | False        | erase        |  0.18  | data/projs/singleton_8852397c03/base.z3/5aca0667.smt2 |
| 200 | mariposa_qid_39  |           107 | False        | inst_replace |  0.16  | data/projs/singleton_8852397c03/base.z3/bdb7bfd5.smt2 |
| 199 | mariposa_qid_39  |        

In [8]:
# print the tested report
sorted_tested = report.tested.sort_values(by=["time"])
print(tabulate(sorted_tested, headers='keys', tablefmt='psql'))

+-----+------------------+--------------+----------+--------+-------------------------------------------------------+
|     | qname            | action       | result   |   time | edit_path                                             |
|-----+------------------+--------------+----------+--------+-------------------------------------------------------|
| 266 | mariposa_qid_99  | inst_keep    | unknown  |  0.102 | data/projs/singleton_8852397c03/base.z3/b8c0a996.smt2 |
|  68 | mariposa_qid_13  | inst_keep    | unknown  |  0.104 | data/projs/singleton_8852397c03/base.z3/371b6885.smt2 |
|  49 | mariposa_qid_264 | inst_replace | unknown  |  0.11  | data/projs/singleton_8852397c03/base.z3/269d21bd.smt2 |
|  99 | mariposa_qid_246 | inst_replace | unknown  |  0.11  | data/projs/singleton_8852397c03/base.z3/523aa572.smt2 |
|  29 | mariposa_qid_474 | skolemize    | unknown  |  0.111 | data/projs/singleton_8852397c03/base.z3/181dbce3.smt2 |
|  43 | mariposa_qid_85  | skolemize    | unknown  |  0.

In [9]:
print(tabulate(report.stabilized.sort_values(by="qname"), headers='keys', tablefmt='psql'))

+----+------------------+--------------+-------------------------------------------------------+
|    | qname            | action       | edit_path                                             |
|----+------------------+--------------+-------------------------------------------------------|
|  0 | mariposa_qid_232 | inst_replace | data/projs/singleton_8852397c03/base.z3/0c641e7e.smt2 |
|  1 | mariposa_qid_232 | inst_keep    | data/projs/singleton_8852397c03/base.z3/a5d59327.smt2 |
+----+------------------+--------------+-------------------------------------------------------+


In [8]:
proof = edi
# in_degrees = proof_graph.in_degree()
# in_degrees = sorted(in_degrees, key=lambda x: x[1], reverse=True)

# parent_in_degrees = {}
# qnames = {}

# for nr, degree in in_degrees:
#     node = proof.lookup_node(nr)
#     if node.name != "quant-inst":
#         continue
#     assert degree == 1
#     # proof.pprint_node(node, 3)
#     qid = proof.get_qname_of_inst_ref(nr)
#     parent = list(proof_graph.predecessors(nr))[0]
#     parent_in_degrees[parent] = proof_graph.in_degree(parent)
#     qnames[parent] = qid

# parent_in_degrees = sorted(parent_in_degrees.items(), key=lambda x: x[1], reverse=True)

# for nr, degree in parent_in_degrees:
#     node = proof.lookup_node(nr)
#     print(degree, qnames[nr], len(nx.shortest_path(proof_graph, source=proof.root_ref, target=nr)))
#     proof.pprint_node(node, 0)

NameError: name 'edi' is not defined